In [2]:
import pandas as pd 
import re


In [30]:
df = pd.read_csv("/home/luanho/Documents/toeic_test.txt" ,sep="/", names=["row"]).dropna()
df.head(10)

,row
0,Her latest film ....... its object in a very s...
1,(a) acquired (b) got (c) attained (d) controlled
2,The critics ....... to praise the work of this...
3,(a) run (b) continue (c) persist (d) insist
4,The film was the first to show conditions in w...
5,(a) influence (b) hold (c) infect (d) show
6,The only reason for them going to the cinema o...
7,(a) internment (b) involvement (c) engrossment...
8,The director's intention in making the film wa...
9,(a) report (b) reprehend (c) represent (d) com...


In [31]:


def transform_row(row):
    # Xóa số dòng ở đầu câu
    row = re.sub(r"^[0-9\.]+", "", row)
    
    # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    row = re.sub(r"[\.,\?]+$", "", row)
    
    # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ")
    
    row = row.strip()
    return row 

df["row"] = df.row.apply(transform_row)
df.head(10)

,row
0,Her latest film its object in a very s...
1,(a) acquired (b) got (c) attained (d) controlled
2,The critics to praise the work of this...
3,(a) run (b) continue (c) persist (d) insist
4,The film was the first to show conditions in w...
5,(a) influence (b) hold (c) infect (d) show
6,The only reason for them going to the cinema o...
7,(a) internment (b) involvement (c) engrossment...
8,The director s intention in making the film wa...
9,(a) report (b) reprehend (c) represent (d) com...


In [32]:


from nltk import ngrams

def kieu_ngram(string, n=1):
    gram_str = list(ngrams(string.split(), n))
    return [ " ".join(gram).lower() for gram in gram_str ]

df["1gram"] = df.row.apply(lambda t: kieu_ngram(t, 1))
df["2gram"] = df.row.apply(lambda t: kieu_ngram(t, 2))

df.head(10)



,row,1gram,2gram
0,Her latest film its object in a very s...,"[her, latest, film, its, object, in, a, very, ...","[her latest, latest film, film its, its object..."
1,(a) acquired (b) got (c) attained (d) controlled,"[(a), acquired, (b), got, (c), attained, (d), ...","[(a) acquired, acquired (b), (b) got, got (c),..."
2,The critics to praise the work of this...,"[the, critics, to, praise, the, work, of, this...","[the critics, critics to, to praise, praise th..."
3,(a) run (b) continue (c) persist (d) insist,"[(a), run, (b), continue, (c), persist, (d), i...","[(a) run, run (b), (b) continue, continue (c),..."
4,The film was the first to show conditions in w...,"[the, film, was, the, first, to, show, conditi...","[the film, film was, was the, the first, first..."
5,(a) influence (b) hold (c) infect (d) show,"[(a), influence, (b), hold, (c), infect, (d), ...","[(a) influence, influence (b), (b) hold, hold ..."
6,The only reason for them going to the cinema o...,"[the, only, reason, for, them, going, to, the,...","[the only, only reason, reason for, for them, ..."
7,(a) internment (b) involvement (c) engrossment...,"[(a), internment, (b), involvement, (c), engro...","[(a) internment, internment (b), (b) involveme..."
8,The director s intention in making the film wa...,"[the, director, s, intention, in, making, the,...","[the director, director s, s intention, intent..."
9,(a) report (b) reprehend (c) represent (d) com...,"[(a), report, (b), reprehend, (c), represent, ...","[(a) report, report (b), (b) reprehend, repreh..."


In [33]:
df["context"] = df["1gram"] + df["2gram"]
train_data = df.context.tolist()
print (len(train_data))

237


In [7]:
df["context"].head(17)

0     [her, latest, film, its, object, in, a, very, ...
1     [(a), acquired, (b), got, (c), attained, (d), ...
2     [the, critics, to, praise, the, work, of, this...
3     [(a), run, (b), continue, (c), persist, (d), i...
4     [the, film, was, the, first, to, show, conditi...
5     [(a), influence, (b), hold, (c), infect, (d), ...
6     [the, only, reason, for, them, going, to, the,...
7     [(a), internment, (b), involvement, (c), engro...
8     [the, director, s, intention, in, making, the,...
9     [(a), report, (b), reprehend, (c), represent, ...
10    [the, fans, outside, the, cinema, on, the, fir...
11                                               [home]
12    [(a), dispose, (b), displace, (c), discover, (...
13    [the, delay, in, the, film, to, the, general, ...
14    [(a), replacing, (b), repeating, (c), releasin...
15    [the, theme, of, the, second, film, is, quite,...
16    [(a), construction, (b), continuation, (c), co...
Name: context, dtype: object

In [8]:

import gensim.models.word2vec as w2v
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)    
model = w2v.Word2Vec(train_data, size=100, window=5, min_count=3, workers=2, sg=1)

Using TensorFlow backend.
2017-10-08 21:56:25,037 : INFO : collecting all words and their counts
2017-10-08 21:56:25,040 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-10-08 21:56:25,043 : INFO : collected 1780 word types from a corpus of 2951 raw words and 134 sentences
2017-10-08 21:56:25,045 : INFO : Loading a fresh vocabulary
2017-10-08 21:56:25,049 : INFO : min_count=3 retains 146 unique words (8% of original 1780, drops 1634)
2017-10-08 21:56:25,050 : INFO : min_count=3 leaves 1194 word corpus (40% of original 2951, drops 1757)
2017-10-08 21:56:25,053 : INFO : deleting the raw counts dictionary of 1780 items
2017-10-08 21:56:25,056 : INFO : sample=0.001 downsamples 94 most-common words
2017-10-08 21:56:25,058 : INFO : downsampling leaves estimated 572 word corpus (47.9% of prior 1194)
2017-10-08 21:56:25,059 : INFO : estimated required memory for 146 words and 100 dimensions: 189800 bytes
2017-10-08 21:56:25,061 : INFO : resetting layer weights
20

In [9]:
model.similar_by_word("her")

2017-10-08 21:56:47,928 : INFO : precomputing L2-norms of word weight vectors


[('the', 0.9766465425491333),
 ('and', 0.9750859141349792),
 ('you', 0.9733900427818298),
 ('this', 0.9725053906440735),
 ('to', 0.9724677205085754),
 ('was', 0.9715151190757751),
 ('film', 0.9714429378509521),
 ('is', 0.9703409075737),
 ('in', 0.9699873328208923),
 ('will', 0.9683954119682312)]

In [12]:
model.save('/home/luanho/Documents/mymodels')


2017-10-08 22:09:07,953 : INFO : saving Word2Vec object under /home/luanho/Documents/mymodels, separately None
2017-10-08 22:09:07,956 : INFO : not storing attribute syn0norm
2017-10-08 22:09:07,959 : INFO : not storing attribute cum_table
2017-10-08 22:09:07,965 : INFO : saved /home/luanho/Documents/mymodels


In [28]:
import gensim
new_model = gensim.models.Word2Vec.load('/home/luanho/Documents/mymodels')

2017-10-08 22:31:04,598 : INFO : loading Word2Vec object from /home/luanho/Documents/mymodels
2017-10-08 22:31:04,606 : INFO : loading wv recursively from /home/luanho/Documents/mymodels.wv.* with mmap=None
2017-10-08 22:31:04,608 : INFO : setting ignored attribute syn0norm to None
2017-10-08 22:31:04,609 : INFO : setting ignored attribute cum_table to None
2017-10-08 22:31:04,610 : INFO : loaded /home/luanho/Documents/mymodels


In [29]:
new_model.similar_by_word('will')

2017-10-08 22:31:37,453 : INFO : precomputing L2-norms of word weight vectors


[('to', 0.9809765815734863),
 ('for', 0.9774883985519409),
 ('is', 0.9762264490127563),
 ('you', 0.9761451482772827),
 ('the', 0.9760264158248901),
 ('i', 0.975692868232727),
 ('about', 0.9756275415420532),
 ('was', 0.9756070375442505),
 ('that', 0.9755958914756775),
 ('in', 0.9753056764602661)]